In [178]:
# import libraries
import pandas as pd

# load the dataset big
#movies_df = pd.read_csv('ml-latest/movies.csv', low_memory=False)
#genome_score_df = pd.read_csv('ml-latest/genome-scores.csv', low_memory=False)
#genome_tags_df = pd.read_csv('ml-latest/genome-tags.csv', low_memory=False)
#links_df = pd.read_csv('ml-latest/links.csv', low_memory=False)
#ratings_df = pd.read_csv('ml-latest/ratings.csv', low_memory=False)
#tags_df = pd.read_csv('ml-latest/tags.csv', low_memory=False)

In [179]:
import pandas as pd
# load the dataset small
movies_df = pd.read_csv('ml-small/movies.csv')
ratings_df = pd.read_csv('ml-small/ratings.csv')
links_df = pd.read_csv('ml-small/links.csv')
tags_df = pd.read_csv('ml-small/tags.csv', low_memory=False)

In [180]:
print(movies_df.shape)
print(links_df.shape)
print(ratings_df.shape)
print(tags_df.shape)

(9742, 3)
(9742, 3)
(100836, 4)
(3683, 4)


In [181]:
#quick overview of datasets to look for potential key to merge df
print(movies_df.head())
print("\n")
print(links_df.head())
print("\n")
print(ratings_df.head())
print("\n")
print(tags_df.head())

   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy  


   movieId  imdbId   tmdbId
0        1  114709    862.0
1        2  113497   8844.0
2        3  113228  15602.0
3        4  114885  31357.0
4        5  113041  11862.0


   userId  movieId  rating  timestamp
0       1        1     4.0  964982703
1       1        3     4.0  964981247
2       1        6     4.0  964982224
3       1       47     5.0  964983815
4       1       50     5.0  964982931


   

In [182]:
#drop timestamp from tags_df (avoid double timestamps)
tags_df.drop(columns='timestamp', inplace=True)

#merging all dataframes
full_df = movies_df.merge(ratings_df, left_on='movieId', right_on='movieId')
full_df = full_df.merge(links_df, left_on='movieId', right_on='movieId')
full_df = full_df.merge(tags_df, left_on='movieId', right_on='movieId')
full_df.shape

(233213, 10)

In [183]:
#check for merged df
full_df

,movieId,title,genres,userId_x,rating,timestamp,imdbId,tmdbId,userId_y,tag
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,964982703,114709,862.0,336,pixar
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,964982703,114709,862.0,474,pixar
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,964982703,114709,862.0,567,fun
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,847434962,114709,862.0,336,pixar
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,847434962,114709,862.0,474,pixar
...,...,...,...,...,...,...,...,...,...,...
233208,187595,Solo: A Star Wars Story (2018),Action|Adventure|Children|Sci-Fi,586,5.0,1529899556,3778644,348350.0,62,star wars
233209,193565,Gintama: The Movie (2010),Action|Animation|Comedy|Sci-Fi,184,3.5,1537098554,1636780,71172.0,184,anime
233210,193565,Gintama: The Movie (2010),Action|Animation|Comedy|Sci-Fi,184,3.5,1537098554,1636780,71172.0,184,comedy
233211,193565,Gintama: The Movie (2010),Action|Animation|Comedy|Sci-Fi,184,3.5,1537098554,1636780,71172.0,184,gintama


In [184]:
#information, looking for missing data or wrong dtypes
full_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 233213 entries, 0 to 233212
Data columns (total 10 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   movieId    233213 non-null  int64  
 1   title      233213 non-null  object 
 2   genres     233213 non-null  object 
 3   userId_x   233213 non-null  int64  
 4   rating     233213 non-null  float64
 5   timestamp  233213 non-null  int64  
 6   imdbId     233213 non-null  int64  
 7   tmdbId     233213 non-null  float64
 8   userId_y   233213 non-null  int64  
 9   tag        233213 non-null  object 
dtypes: float64(2), int64(5), object(3)
memory usage: 19.6+ MB


## Create based function

### Scored based on votes and ratings

In [185]:
# calculate average with count
avg_rating_df = full_df.groupby(['title'])['rating'].agg(['mean', 'count'])
avg_rating_df.rename(columns={'mean': 'vote_average', 'count': 'vote_count'}, inplace=True)
avg_rating_df.sort_values('vote_average', ascending=False).head(10)

# C is the mean vote or rating across the whole dataframe
C = avg_rating_df['vote_average'].mean()

# m is the minimum votes or ratings to be listed | using a low value because the dataset is small
m = avg_rating_df['vote_count'].quantile(0.80)
print('Numbers of votes to be in final list:', m)
# weight function (ranked based on number of votes AND score)
def weighted_rating(x, m=m, C=C):
    v = x['vote_count']
    R = x['vote_average']
    return (v/(v+m) * R) + (m/(m+v) * C)

# check the new scores
avg_rating_df = avg_rating_df.copy().loc[avg_rating_df['vote_count'] >= m]
avg_rating_df['score'] = avg_rating_df.apply(weighted_rating, axis=1)

#list of top 10 movies with best "Scores"
avg_rating_df.sort_values('score', ascending=False).head(10)


Numbers of votes to be in final list: 92.0


,vote_average,vote_count,score
title,,,
"Shawshank Redemption, The (1994)",4.429022,1268,4.376483
Fight Club (1999),4.272936,11772,4.268123
Star Wars: Episode IV - A New Hope (1977),4.231076,6526,4.223031
Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1964),4.268041,970,4.214705
"Departed, The (2006)",4.252336,1070,4.204834
"Usual Suspects, The (1995)",4.237745,1224,4.196822
Pulp Fiction (1994),4.197068,55567,4.196168
Schindler's List (1993),4.225000,1540,4.192719
Star Wars: Episode V - The Empire Strikes Back (1980),4.215640,2110,4.192106


In [186]:
#list of top 10 users with most ratings => trending users!
avg_rating_user = full_df.groupby(['userId_x'])['rating'].agg(['mean', 'count'])
avg_rating_user.rename(columns={'mean': 'vote_average', 'count': 'vote_count'}, inplace=True)
avg_rating_user.sort_values('vote_count', ascending=False).head(10)

,vote_average,vote_count
userId_x,,
474,3.778004,2455
414,4.134016,2343
599,3.480476,2100
68,3.183696,1791
610,4.310994,1701
274,3.811111,1665
249,4.188685,1635
448,3.636508,1575
380,4.250651,1536


### Create Combinations and occurance of pairwise!

In [187]:
from itertools import permutations
# create a function that makes combinations of the movie that a user (x) reviewed
def create_combinations(x):
    combinations = pd.DataFrame(list(permutations(x.values, 2)), columns=['item_a', 'item_b'])
    return combinations


In [188]:
#number of users
print(len(full_df['userId_x'].unique()))

#number of movies
len(full_df['title'].unique())

610


1554

In [189]:
#avoid users with only 60 review to measure impact 
avg_user = full_df.groupby(['userId_x']).size().reset_index(name='counts')
avg_user = avg_user[avg_user['counts'] > 100]
avg_user.sort_values(by='counts', ascending=False)

,userId_x,counts
473,474,2455
413,414,2343
598,599,2100
67,68,1791
609,610,1701
...,...,...
454,455,105
270,271,104
328,329,103
184,185,102


In [190]:
# use the create_combinations function
movie_combinations = full_df.groupby('userId_x')['title'].apply(create_combinations)
movie_combinations = movie_combinations.reset_index(drop=True)
combi_count = movie_combinations.groupby(['item_a', 'item_b']).size()
combi_count = combi_count.to_frame(name='size').reset_index()

#remove duplicates (same movie iteam_a and b)
combi_count['equal'] = combi_count['item_a'] == combi_count["item_b"]
combi_count = combi_count[combi_count['equal'] == False]
combi_count.drop(columns='equal', inplace=True) #remove this ugly name!

#sort by size!
combi_count.sort_values('size', ascending=False)

,item_a,item_b,size
1356930,Pulp Fiction (1994),Fight Club (1999),1544292
553767,Fight Club (1999),Pulp Fiction (1994),1544292
1357781,Pulp Fiction (1994),Star Wars: Episode IV - A New Hope (1977),724724
1643533,Star Wars: Episode IV - A New Hope (1977),Pulp Fiction (1994),724724
1009061,Léon: The Professional (a.k.a. The Professiona...,Pulp Fiction (1994),684180
...,...,...,...
685262,"Great Santini, The (1979)",Being Julia (2004),1
1518035,Shadowlands (1993),Billy Elliot (2000),1
1518034,Shadowlands (1993),"Bill Cosby, Himself (1983)",1
1518032,Shadowlands (1993),Bill & Ted's Bogus Journey (1991),1


In [191]:
#Similarities based on a text search for example and a default parameter
search_box = "Star Wars"
minium_size = 30 #users who watched item_a, watched item_b

test_names = combi_count[( combi_count['item_a'].str.contains(search_box) ) & (combi_count['size'] > minium_size)]
test_names.sort_values(by='size', ascending=False)

,item_a,item_b,size
1643533,Star Wars: Episode IV - A New Hope (1977),Pulp Fiction (1994),724724
1645074,Star Wars: Episode V - The Empire Strikes Back...,Pulp Fiction (1994),251590
1642900,Star Wars: Episode IV - A New Hope (1977),Fight Club (1999),188136
1642474,Star Wars: Episode IV - A New Hope (1977),2001: A Space Odyssey (1968),90610
1643263,Star Wars: Episode IV - A New Hope (1977),Léon: The Professional (a.k.a. The Professiona...,77350
...,...,...,...
1584789,Solo: A Star Wars Story (2018),"Shawshank Redemption, The (1994)",32
1640940,Star Wars: Episode III - Revenge of the Sith (...,10 Cloverfield Lane (2016),32
1584651,Solo: A Star Wars Story (2018),Old Boy (2003),32
1640883,Star Wars: Episode I - The Phantom Menace (1999),When We Were Kings (1996),32


In [193]:
#test some recommendation based of ocurrance

#variable defined by user 
#film_name = input('Type the name of your movie:')

#to run 1st time without pausing
film_name = "Star Wars: Episode VI - Return of the Jedi (1983)"
common_movies = combi_count[combi_count['item_a'] == film_name].sort_values("size", ascending=False).head(5)
#common_movies = common_movies['item_b']
#common_movies = common_movies.rename(columns={"item_b": "Movie_Names"})
common_movies = common_movies.reset_index(drop=True)

print("These are the 5 movies that users who watched {}, rated:".format(film_name))
#merge rating informations (Dataset create before and using list of name as a input)

#remember scores = number of votes and rates!!
scores = avg_rating_df["score"].loc[common_movies['item_b']]
common_movies['score'] = scores.values
#common_movies.drop(columns='equal', inplace=True)
common_movies.sort_values("score", ascending=False).head(5)


These are the 5 movies that users who watched Star Wars: Episode VI - Return of the Jedi (1983), rated:


,item_a,item_b,size,score
1,Star Wars: Episode VI - Return of the Jedi (1983),Fight Club (1999),16848,4.268123
2,Star Wars: Episode VI - Return of the Jedi (1983),Star Wars: Episode IV - A New Hope (1977),13650,4.223031
0,Star Wars: Episode VI - Return of the Jedi (1983),Pulp Fiction (1994),66246,4.196168
4,Star Wars: Episode VI - Return of the Jedi (1983),Léon: The Professional (a.k.a. The Professiona...,7665,4.011695
3,Star Wars: Episode VI - Return of the Jedi (1983),2001: A Space Odyssey (1968),9348,3.889611


### Similarities between Genres or Users

In [194]:
# remove duplicates
full_df2 = full_df.copy().drop_duplicates(subset=['title'])

# cast to list
full_df2['genres'] = full_df2.genres.str.split('|').tolist()
#full_df2['genres'].head(10)

# explode into rows
full_df2 = full_df2.explode('genres')
#full_df2.head(5)

# let's make a crosstab
movies_category = pd.crosstab(full_df2['title'], full_df2['genres'])
movies_category


genres,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
title,,,,,,,,,,,,,,,,,,,,
(500) Days of Summer (2009),0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0
...And Justice for All (1979),0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0
10 Cloverfield Lane (2016),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
10 Things I Hate About You (1999),0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0
101 Dalmatians (1996),0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Zero Dark Thirty (2012),0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0
Zombieland (2009),0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0
Zoolander (2001),0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [195]:
#user similarities vs genres

# remove duplicates
full_df3 = full_df.copy().drop_duplicates(subset=['title'])

full_df3['genres'] = full_df3['genres'].str.split('|').tolist()

# explode into rows
full_df3 = full_df3.explode('genres')
#full_df3.head(5)

# let's make a crosstab
users_category = pd.crosstab(full_df3['userId_x'], full_df3['genres'])
users_category


genres,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
userId_x,,,,,,,,,,,,,,,,,,,,
1,0,41,37,14,16,39,28,0,40,20,1,7,0,10,12,15,25,28,9,3
2,0,8,2,0,0,3,8,1,12,0,0,1,4,0,2,1,3,6,1,1
3,0,1,1,2,2,6,1,0,11,1,0,0,0,1,0,5,0,0,3,0
4,0,5,8,2,3,36,14,1,62,3,3,2,1,5,11,30,2,16,5,4
5,0,4,4,2,5,5,3,0,11,2,0,0,2,2,0,5,1,2,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
514,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
560,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
567,1,0,2,3,0,2,0,0,9,2,0,4,0,0,2,2,2,3,0,0


In [196]:
#Or we can have users by movies similarities

# remove duplicates
full_df4 = full_df.copy()

# let's make a crosstab
users_movies = pd.crosstab(full_df4['userId_x'], full_df3['title'])
users_movies


title,(500) Days of Summer (2009),...And Justice for All (1979),10 Cloverfield Lane (2016),10 Things I Hate About You (1999),101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),"11'09""01 - September 11 (2002)",12 Angry Men (1957),127 Hours (2010),13 Going on 30 (2004),...,Young Frankenstein (1974),Z (1969),Zack and Miri Make a Porno (2008),Zelary (2003),Zelig (1983),Zero Dark Thirty (2012),Zombieland (2009),Zoolander (2001),Zulu (1964),eXistenZ (1999)
userId_x,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,...,2,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,3,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
514,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
560,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
567,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Distances Similarities (Jaccard)

In [197]:
# import necessary stuff
from sklearn.metrics import jaccard_score
from scipy.spatial.distance import pdist, squareform

#movies distances based on genres
# calculate the distances
jaccard_distances = pdist(movies_category.values, metric='jaccard')
square_jaccard_distances = squareform(jaccard_distances)

#square_jaccard_distances
jaccard_similarity_array = 1 - square_jaccard_distances
#jaccard_similarity_array

distance_df_movies_category = pd.DataFrame(jaccard_similarity_array, index=movies_category.index, columns=movies_category.index)
distance_df_movies_category


title,(500) Days of Summer (2009),...And Justice for All (1979),10 Cloverfield Lane (2016),10 Things I Hate About You (1999),101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),"11'09""01 - September 11 (2002)",12 Angry Men (1957),127 Hours (2010),13 Going on 30 (2004),...,Young Frankenstein (1974),Z (1969),Zack and Miri Make a Porno (2008),Zelary (2003),Zelig (1983),Zero Dark Thirty (2012),Zombieland (2009),Zoolander (2001),Zulu (1964),eXistenZ (1999)
title,,,,,,,,,,,,,,,,,,,,,
(500) Days of Summer (2009),1.000000,0.250000,0.000000,0.666667,0.200000,0.0,0.333333,0.333333,0.200000,0.500000,...,0.250000,0.200000,1.000000,0.666667,0.333333,0.200000,0.200000,0.333333,0.20,0.000000
...And Justice for All (1979),0.250000,1.000000,0.500000,0.000000,0.000000,0.0,0.500000,0.500000,0.666667,0.000000,...,0.000000,0.666667,0.250000,0.333333,0.000000,0.666667,0.000000,0.000000,0.25,0.250000
10 Cloverfield Lane (2016),0.000000,0.500000,1.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.333333,0.000000,...,0.000000,0.333333,0.000000,0.000000,0.000000,0.333333,0.000000,0.000000,0.00,0.333333
10 Things I Hate About You (1999),0.666667,0.000000,0.000000,1.000000,0.250000,0.0,0.000000,0.000000,0.000000,0.666667,...,0.333333,0.000000,0.666667,0.333333,0.500000,0.000000,0.250000,0.500000,0.00,0.000000
101 Dalmatians (1996),0.200000,0.000000,0.000000,0.250000,1.000000,0.5,0.000000,0.000000,0.200000,0.200000,...,0.250000,0.000000,0.200000,0.000000,0.333333,0.000000,0.200000,0.333333,0.00,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Zero Dark Thirty (2012),0.200000,0.666667,0.333333,0.000000,0.000000,0.0,0.333333,0.333333,0.500000,0.000000,...,0.000000,0.500000,0.200000,0.250000,0.000000,1.000000,0.200000,0.000000,0.50,0.500000
Zombieland (2009),0.200000,0.000000,0.000000,0.250000,0.200000,0.0,0.000000,0.000000,0.000000,0.200000,...,0.250000,0.000000,0.200000,0.000000,0.333333,0.200000,1.000000,0.333333,0.20,0.200000
Zoolander (2001),0.333333,0.000000,0.000000,0.500000,0.333333,0.0,0.000000,0.000000,0.000000,0.333333,...,0.500000,0.000000,0.333333,0.000000,1.000000,0.000000,0.333333,1.000000,0.00,0.000000


In [198]:
#list top 10 similar movies:
movie_name = "Zombieland (2009)"

distance_df_movies_category[movie_name].sort_values(ascending=False).head(11) #because the first element is the movie

title
Zombieland (2009)                                       1.000000
Game Night (2018)                                       0.750000
Tucker & Dale vs Evil (2010)                            0.666667
Rare Exports: A Christmas Tale (Rare Exports) (2010)    0.666667
Jaws 3-D (1983)                                         0.666667
Jaws (1975)                                             0.666667
Get Smart (2008)                                        0.666667
The Interview (2014)                                    0.666667
Once Bitten (1985)                                      0.666667
Observe and Report (2009)                               0.666667
My Demon Lover (1987)                                   0.666667
Name: Zombieland (2009), dtype: float64

In [199]:
# calculate the distances | change a bit to be easier to change the cross_tab input

#users distances based on genres/categories
cross_tab = users_category
jaccard_distances = pdist(cross_tab.values, metric='jaccard')
square_jaccard_distances = squareform(jaccard_distances)

#square_jaccard_distances
jaccard_similarity_array = 1 - square_jaccard_distances
#jaccard_similarity_array

distance_df = pd.DataFrame(jaccard_similarity_array, index=cross_tab.index, columns=cross_tab.index)
distance_df

userId_x,1,2,3,4,5,6,7,8,9,10,...,462,465,474,477,509,514,560,567,594,606
userId_x,,,,,,,,,,,,,,,,,,,,,
1,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.055556,0.000000,0.058824,0.000000,...,0.000000,0.000000,0.052632,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000
2,0.0,1.000000,0.000000,0.052632,0.000000,0.055556,0.052632,0.000000,0.000000,0.111111,...,0.066667,0.000000,0.000000,0.200000,0.000000,0.071429,0.00,0.117647,0.071429,0.071429
3,0.0,0.000000,1.000000,0.052632,0.214286,0.066667,0.058824,0.000000,0.066667,0.071429,...,0.083333,0.000000,0.055556,0.000000,0.090909,0.000000,0.00,0.000000,0.000000,0.000000
4,0.0,0.052632,0.052632,1.000000,0.052632,0.105263,0.052632,0.000000,0.000000,0.157895,...,0.000000,0.000000,0.157895,0.052632,0.000000,0.000000,0.00,0.050000,0.000000,0.000000
5,0.0,0.000000,0.214286,0.052632,1.000000,0.062500,0.058824,0.000000,0.133333,0.214286,...,0.000000,0.000000,0.052632,0.000000,0.000000,0.000000,0.00,0.062500,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
514,0.0,0.071429,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.100000,0.000000,1.000000,0.00,0.000000,0.000000,0.000000
560,0.0,0.000000,0.000000,0.000000,0.000000,0.066667,0.000000,0.333333,0.000000,0.000000,...,0.000000,0.000000,0.055556,0.100000,0.333333,0.000000,1.00,0.000000,0.250000,0.000000
567,0.0,0.117647,0.000000,0.050000,0.062500,0.000000,0.000000,0.000000,0.000000,0.058824,...,0.000000,0.000000,0.000000,0.076923,0.000000,0.000000,0.00,1.000000,0.000000,0.000000


In [200]:
#list top 10 similar users: 
user_number = 474 #top 1 user with most ratings
distance_df[user_number].sort_values(ascending=False).head(11) #because the first element is the user

userId_x
474    1.000000
17     0.157895
4      0.157895
105    0.111111
45     0.111111
371    0.111111
509    0.111111
82     0.111111
7      0.105263
64     0.105263
103    0.105263
Name: 474, dtype: float64

In [201]:
# calculate the distances | change a bit to be easier to change the cross_tab input

#users distances based on Movies
cross_tab = users_movies
jaccard_distances = pdist(cross_tab.values, metric='jaccard')
square_jaccard_distances = squareform(jaccard_distances)

#square_jaccard_distances
jaccard_similarity_array = 1 - square_jaccard_distances
#jaccard_similarity_array

distance_df = pd.DataFrame(jaccard_similarity_array, index=cross_tab.index, columns=cross_tab.index)
distance_df

userId_x,1,2,3,4,5,6,7,8,9,10,...,462,465,474,477,509,514,560,567,594,606
userId_x,,,,,,,,,,,,,,,,,,,,,
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
514,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
560,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
567,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [202]:
users_movies.loc[users_movies.index == 474]

title,(500) Days of Summer (2009),...And Justice for All (1979),10 Cloverfield Lane (2016),10 Things I Hate About You (1999),101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),"11'09""01 - September 11 (2002)",12 Angry Men (1957),127 Hours (2010),13 Going on 30 (2004),...,Young Frankenstein (1974),Z (1969),Zack and Miri Make a Porno (2008),Zelary (2003),Zelig (1983),Zero Dark Thirty (2012),Zombieland (2009),Zoolander (2001),Zulu (1964),eXistenZ (1999)
userId_x,,,,,,,,,,,,,,,,,,,,,
474,0,0,0,0,0,0,1,0,0,0,...,0,0,0,2,0,0,0,0,0,0


In [203]:
#list top 10 similar users: As we can see similarities with users based on movies has a too sparce matrix
user_number = 474 #top 1 user with most ratings
distance_df[user_number].sort_values(ascending=False).head(11) #because the first element is the user


userId_x
474    1.0
606    0.0
64     0.0
84     0.0
82     0.0
79     0.0
74     0.0
70     0.0
68     0.0
66     0.0
63     0.0
Name: 474, dtype: float64

### Cosine Similarities

In [204]:
full_df.head()

,movieId,title,genres,userId_x,rating,timestamp,imdbId,tmdbId,userId_y,tag
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,964982703,114709,862.0,336,pixar
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,964982703,114709,862.0,474,pixar
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,964982703,114709,862.0,567,fun
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,847434962,114709,862.0,336,pixar
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,847434962,114709,862.0,474,pixar


In [205]:
#select a subset of fields only
smaller_selection = full_df.groupby(['title', 'tag', 'genres']).size().reset_index(name='counts')
#smaller_selection = smaller_selection[smaller_selection['counts'] > 2]
smaller_selection = smaller_selection.reset_index().drop_duplicates()
smaller_selection.shape
smaller_selection


,index,title,tag,genres,counts
0,0,(500) Days of Summer (2009),Funny,Comedy|Drama|Romance,42
1,1,(500) Days of Summer (2009),Zooey Deschanel,Comedy|Drama|Romance,42
2,2,(500) Days of Summer (2009),artistic,Comedy|Drama|Romance,42
3,3,(500) Days of Summer (2009),humorous,Comedy|Drama|Romance,42
4,4,(500) Days of Summer (2009),inspiring,Comedy|Drama|Romance,42
...,...,...,...,...,...
3553,3553,Zoolander (2001),comedy,Comedy,54
3554,3554,Zoolander (2001),goofy,Comedy,54
3555,3555,Zoolander (2001),mindless one liners,Comedy,54
3556,3556,Zulu (1964),Africa,Action|Drama|War,4


In [206]:
#example of how many tags a movie have
smaller_selection[smaller_selection['title']=="Toy Story (1995)"].groupby('tag')['title'].count()

tag
fun      1
pixar    1
Name: title, dtype: int64

In [207]:
#need to group tags in one row
smaller_selection_tag = smaller_selection.copy()
# concatenate the string 
smaller_selection_tag['tag'] = smaller_selection_tag.groupby(['title'])['tag'].transform(lambda x : ' '.join(x)) 
  
# drop duplicate data

#smaller_selection_tag.drop(columns=['index', 'counts'], inplace=True)
smaller_selection_tag = smaller_selection_tag.drop_duplicates(subset=['title'])  
smaller_selection_tag.reset_index(inplace=True)

# show the dataframe 
smaller_selection_tag

,level_0,index,title,tag,genres,counts
0,0,0,(500) Days of Summer (2009),Funny Zooey Deschanel artistic humorous inspir...,Comedy|Drama|Romance,42
1,8,8,...And Justice for All (1979),lawyers,Drama|Thriller,3
2,9,9,10 Cloverfield Lane (2016),creepy suspense,Thriller,14
3,11,11,10 Things I Hate About You (1999),Shakespeare sort of,Comedy|Romance,54
4,12,12,101 Dalmatians (1996),dogs remake,Adventure|Children|Comedy,47
...,...,...,...,...,...,...
1549,3539,3539,Zero Dark Thirty (2012),Afghanistan American propaganda assassination ...,Action|Drama|Thriller,14
1550,3544,3544,Zombieland (2009),Bill Murray Emma Stone Jesse Eisenberg Woody H...,Action|Comedy|Horror,53
1551,3550,3550,Zoolander (2001),David Bowie Will Ferrell ben stiller comedy go...,Comedy,54
1552,3556,3556,Zulu (1964),Africa,Action|Drama|War,4


In [208]:
# import TfidfVectorize
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words='english')

smaller_selection_tag['tag'] = smaller_selection_tag['tag'].fillna('')

tfidf_matrix = tfidf.fit_transform(smaller_selection_tag['tag'])

tfidf_matrix.shape


(1554, 1671)

In [209]:
from sklearn.metrics.pairwise import linear_kernel

cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
cosine_sim

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [210]:
indices = pd.Series(smaller_selection_tag.index, index=smaller_selection_tag['title']).drop_duplicates()
indices

title
(500) Days of Summer (2009)             0
...And Justice for All (1979)           1
10 Cloverfield Lane (2016)              2
10 Things I Hate About You (1999)       3
101 Dalmatians (1996)                   4
                                     ... 
Zero Dark Thirty (2012)              1549
Zombieland (2009)                    1550
Zoolander (2001)                     1551
Zulu (1964)                          1552
eXistenZ (1999)                      1553
Length: 1554, dtype: int64

In [211]:
indices['Zoolander (2001)']


1551

In [212]:
def get_recommendations(title, cosine_sim=cosine_sim):
    # Get the index of the movie that matches the title
    idx = indices[title]

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:11]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return smaller_selection_tag.iloc[movie_indices]

In [213]:
#variable defined by user 
#film_name = input('Type the name of your movie:')
film_name = "101 Dalmatians (1996)"

print('Your movie tags: \n{}'.format(smaller_selection_tag["tag"].iloc[indices[film_name]]))
print('Your movie genre: \n{}'.format(smaller_selection_tag["genres"].iloc[indices[film_name]]))
get_recommendations(film_name)

Your movie tags: 
dogs remake
Your movie genre: 
Adventure|Children|Comedy


,level_0,index,title,tag,genres,counts
1432,3311,3311,Turner & Hooch (1989),dogs,Comedy|Crime,13
581,1378,1378,"Haunting, The (1999)",remake,Horror|Thriller,13
1161,2703,2703,Sabrina (1995),remake,Comedy|Romance,54
1452,3373,3373,"Vanishing, The (1993)",remake,Mystery|Thriller,4
135,358,358,Best in Show (2000),Dogs satire,Comedy,53
430,1039,1039,Father of the Bride (1991),remake wedding,Comedy,13
431,1041,1041,Father of the Bride Part II (1995),pregnancy remake,Comedy,49
1231,2851,2851,Silk Stockings (1957),Ninotchka remake,Musical,1
1543,3531,3531,You've Got Mail (1998),e-mail remake,Comedy|Romance,50
1358,3139,3139,The Count of Monte Cristo (2002),Dumas remake revenge,Action|Adventure|Drama|Thriller,32


In [214]:
#after see the results, only tags dont see to work => A comedy movie and a Horror movie dosent seems right match... So lets try to mix with genre as well!


# Ideias

1. Which movies are most watched(rated) all the time? - OK done first part
2. Who is the user that watched the most number of movies? - - OK done first part
3. Graph of Genres (barplot) with avg ratings, and top movie by each genre!
4. Genres that a user NEVER watch but is well rated -> include diversity!
5. Clusterize all users: detect similar preferences! It is possible to aquire or simulate "random" demographics: age, sex,... ? - OK done first part Cluster users by similaritity in genre and movies rated

# Basic Functions

In [215]:
def expand_colum(df, column_name, field='title'):
    """ transform a colum with stacked values in a crosstab colum """
    # remove duplicates
    df2 = df.drop_duplicates(subset=['title'])

    # get rid of those pesky []'s and '
    #full_df2['genres2'] = full_df2.genres.apply(lambda x: x.replace('|', ''))
    #full_df2['genres2'] = full_df2.genres.apply(lambda x: x.replace(']', ''))
    #full_df2['genres2'] = full_df2.genres.apply(lambda x: x.replace("'", ""))

    # cast to list
    if df2[column_name].dtype == "int64":
        df2[column_name] = df2[column_name].tolist()
    else:
        df2[column_name] = df2[column_name].str.split('|').tolist()
    # explode into rows
    df = df2.explode(column_name)
    
    # let's make a crosstab
    df_column = pd.crosstab(df[field], df[column_name])
    df_column
    
    #return one df with columns by row (util to average statistics for example)
    #return a crosstab table
    return df, df_column

def calc_distance(cross_df):
    """ receive a crosstab df and return a distance dataframe"""
    # import necessary stuff
    from sklearn.metrics import jaccard_score
    from scipy.spatial.distance import pdist, squareform

    # calculate the distances
    jaccard_distances = pdist(cross_df.values, metric='jaccard')
    square_jaccard_distances = squareform(jaccard_distances)

    #square_jaccard_distances
    jaccard_similarity_array = 1 - square_jaccard_distances
    #jaccard_similarity_array
    distance_df = pd.DataFrame(jaccard_similarity_array, index=cross_df.index, columns=cross_df.index)
    return distance_df

def get_similarities(distance_df, col_name, numbers=5):
    """receive a distance df and film_name/user_number, and return a list of "numbers" films"""
    #variable defined by user 
    #film_name = input('Type the name of your movie:')
    print('Top {} Similar with {}'.format(numbers,col_name))
    print(distance_df[col_name].sort_values(ascending=False).head(numbers))
    
def create_combinations(x):
    """to be used inside another functions as apply method"""
    from itertools import permutations

    # create a function that makes combinations of the movie that a user (x) reviewed
    
    combinations = pd.DataFrame(list(permutations(x.values, 2)), columns=['item_a', 'item_b'])
    return combinations

def create_combination_df(df, col_filter='userId_x', col_group='title'):
    # use the create_combinations function
    movie_combinations = df.groupby(col_filter)[col_group].apply(create_combinations)

    movie_combinations = movie_combinations.reset_index(drop=True)
    combi_count = movie_combinations.groupby(['item_a', 'item_b']).size()
    combi_count = combi_count.to_frame(name='size').reset_index()

    #remove duplicates (same movie iteam_a and b)
    combi_count['equal'] = combi_count['item_a'] == combi_count["item_b"]
    combi_count = combi_count[combi_count['equal'] == False]

    combi_count.sort_values('size', ascending=False)
    return combi_count

def cosine_similatirity(df, col_filter='tag'):
    
    from sklearn.feature_extraction.text import TfidfVectorizer
    from sklearn.metrics.pairwise import linear_kernel
    #create object and define stop words language
    tfidf = TfidfVectorizer(stop_words='english')

    #fill na if exists
    df[col_filter] = df[col_filter].fillna('')

    #transform based on object tfidf
    tfidf_matrix = tfidf.fit_transform(df[col_filter])
    
    #apply functions
    cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
    indices = pd.Series(df.index, index=df['title']).drop_duplicates()
    
    #return indices and df of similarities
    return indices, cosine_sim


### NEED FIX #####
def get_recommendations_cos(title, df, indices, cosine_sim):
    # Get the index of the movie that matches the title
    idx = indices[title]

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:11]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]
    print('Your movie tags: \n{}'.format(df["tag"].iloc[df[title]]))
    print('Your movie genre: \n{}'.format(df["genres"].iloc[df[title]]))
    # Return the top 10 most similar movies
    return df.iloc[movie_indices]

def weight_votes(df, group_col, agg_col1='vote_average', agg_col2='vote_count', number=10):
    # calculate average with count
    #avg_rating_df = df.groupby([group_col])[agg_col].agg(['mean', 'count'])
       # avg_rating_df.rename(columns={'mean': 'vote_average', 'count': 'vote_count'}, inplace=True)
    #avg_rating_df.sort_values('vote_average', ascending=False).head(10)

    # C is the mean vote or rating across the whole dataframe
    C = df['vote_average'].mean()

    # m is the minimum votes or ratings to be listed | using a low value because the dataset is small
    m = df['vote_count'].quantile(0.10)

    # weight function (ranked based on number of votes AND score)
    def weighted_rating(x, m=m, C=C):
        v = x['vote_count']
        R = x['vote_average']
        return (v/(v+m) * R) + (m/(m+v) * C)

    # check the new scores
    avg_rating_df = df.copy().loc[df['vote_count'] >= m]
    avg_rating_df['score'] = avg_rating_df.apply(weighted_rating, axis=1)
    avg_rating_df = avg_rating_df.sort_values('score', ascending=False).head(number)
    return avg_rating_df

def concat_field(df, col_name):
    """ receive a df and a column name, concat multiple rows on that column, for example all rows with different tags"""
    #need to group columns in one row #example tag
   
    # concatenate the string 
    df[col_name] = df.groupby(['title'])[col_name].transform(lambda x : ' '.join(x)) 

    # drop duplicate data | not using because i prefer df with all rows, so u can made the statics better

    #smaller_selection_tag.drop(columns=['index', 'counts'], inplace=True)
    #smaller_selection_tag = smaller_selection_tag.drop_duplicates(subset=['tag'])  
    #smaller_selection_tag.reset_index(inplace=True)

    # show the dataframe 
    return smaller_selection_tag

def create_date_of_ratings(df, col_name):
    pass

# Rating Movies and how update the database

In [216]:
#create function to rate a movie and update database
def rating_movie(df, title, rating):
    pass

#update all tables metrics: like distances tables, main database, scores...
#it should run on backgroung to avoid user experience being compromised
def update_tables():
    pass


In [ ]:
x1, x2 = expand_colum(full_df, "genres")
x2

<ipython-input-215-d1f3d5b1bf1e>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2[column_name] = df2[column_name].str.split('|').tolist()


genres,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
title,,,,,,,,,,,,,,,,,,,,
(500) Days of Summer (2009),0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0
...And Justice for All (1979),0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0
10 Cloverfield Lane (2016),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
10 Things I Hate About You (1999),0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0
101 Dalmatians (1996),0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Zero Dark Thirty (2012),0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0
Zombieland (2009),0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0
Zoolander (2001),0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


# Content Based Recommender

### Recommends content similar to the content the user has consumed in the past

In [ ]:
#get top 5 movies by genre | we can use it later to define other parameters

#explode genres column
full_df_genres, full_df_cross = expand_colum(full_df, "genres")

#look for user top genre

full_df_genres

In [ ]:
#Ratings by user per genres
users_genre = full_df_genres.groupby(by=["userId_x","genres"])["rating"].agg(['mean', "count"]).reset_index()
users_genre.rename(columns={'mean': 'vote_average', 'count': 'vote_count'}, inplace=True)
users_genre

In [ ]:
#get top 5 movies by genre | we can use it later to define other parameters
#first look for the highest rank genre for one user
user_number = 10
user_genre = users_genre[ (users_genre['userId_x'] == user_number) ] 

#sort_values by score (composite of rating and number of votes)
user_genre = weight_votes(user_genre, "genres")
print(user_genre)

user_genre.sort_values(by=("score"), ascending=False, inplace=True)

print("Your favorite genre is: {}". format(user_genre["genres"].iloc[0]))
print("Score of this genre: {:.2f}". format(user_genre["score"].iloc[0]))
print("Number of rated movies of this genre: {}". format(user_genre["vote_count"].iloc[0]))
#assign to a variable
genre = user_genre["genres"].iloc[0]

In [ ]:
#filter in data_set movies that he already watched
filter_user = full_df_genres[ (full_df_genres['genres'] == genre) & (full_df_genres['userId_x'] == user_number) ] 
filter_user

In [ ]:
#list of all movies from selected genre
index = full_df_genres[["title", "genres"]][full_df_genres['genres'] == genre]
full_df_genres[["title", "genres"]][full_df_genres['genres'] == genre]

In [ ]:
#list of all movies from selected genre that user DONT watch yet
filter_user_not = full_df_genres[ (full_df_genres['genres'] == genre) & (full_df_genres['userId_x'] != user_number) ] 
filter_user_not = filter_user_not.groupby(by=["title","genres"])["rating"].agg(['mean', "count"]).reset_index()
filter_user_not.rename(columns={'mean': 'vote_average', 'count': 'vote_count'}, inplace=True)

#as dataset is small there is basic one vote by movie
filter_user_not = weight_votes(filter_user_not, "genres", number=1000)
filter_user_not.sort_values(by="score", ascending=False)

In [ ]:
#Crate a ratings by genre
ratings_genre = full_df_genres.groupby(by=["genres"])["rating"].agg(['mean', "count"]).reset_index()
ratings_genre.rename(columns={'mean': 'vote_average', 'count': 'vote_count'}, inplace=True)
ratings_genre = weight_votes(ratings_genre, "genres", number=100)
ratings_genre.sort_values(by="score", ascending=False)

In [ ]:
#Recommend the 10 movies similar to the best movie for this user

In [ ]:
#Select 3 most relevant genres based on user rating (Score = weighted between avg and number of votes)

In [ ]:
#Select 3 most relevant tags based on user rating (Score = weighted between avg and number of votes)

In [ ]:
#recommend 5 movies based on this criterias

In [ ]:
#recommend 5 movies based on this criterias

In [ ]:
#Recommend the 10 movies based to the last movie for this user

# Collaborative Filtering

### Users who are similar to the target user have consumed

In [ ]:
#explode users column to build user similarities
full_df_users, full_df_cross = expand_colum(full_df, "userId_x")
full_df_cross

In [ ]:

user = 10
#select a film to test

#create oone crosstab by genres and other by users similatires
users = "userId_x"
genres = "genres"
x1, df_cross_users = expand_colum(full_df, users) #x1 -> only if we have colum with sepators like "|"
genre_rows, df_cross_genres = expand_colum(full_df, genres)

#create distances matrices
df_dist_users = calc_distance(df_cross_users )
df_dist_genres = calc_distance(df_cross_genres)
df_dist_users



In [ ]:
#get similatires

#top 5 similar users:


In [ ]:
#top 5 movies based on users similarities
print("similatires based user")
users_similarities = get_similarities(df_dist_users, film_name, numbers=5)
print("\n")
print("similatires based on  genre")
genres_similarities = get_similarities(df_dist_genres, film_name, numbers=5)

# Colababorative Filtering v2

How do you determine which users or items are similar to one another?

Given that you know which users are similar, how do you determine the rating that a user would give to an item based on the ratings of similar users?

How do you measure the accuracy of the ratings you calculate?

One important thing to keep in mind is that in an approach based purely on collaborative filtering, the similarity is not calculated using factors like the age of users, genre of the movie, or any other data about users or items. It is calculated only on the basis of the rating (explicit or implicit) a user gives to an item. For example, two users can be considered similar if they give the same ratings to ten movies despite there being a big difference in their age.

## Memory Based

To find the rating R that a user U would give to an item I, the approach includes:

1.Finding users similar to U who have rated the item I 

2.Calculating the rating R based the ratings of users found in the previous step

In [ ]:
# Using Cosine distances

# With the similarity factor S for each user similar to the target user U, you can calculate the weighted average using this formula:
#Sum(Ru*Su)/Sum(Su)


## Model Based

# News Content

### Recomends the lastest / Diverse

In [ ]:
#recommend the TRENDING movie based on other similar users choices


In [ ]:
#recommend the TRENDING movie based on DIVERSITY choices